In [1]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_lg')
df = pd.read_csv('data/train.csv')
df = df.fillna('')

from sklearn.model_selection import train_test_split
df['text'] = df['text'].apply(lambda x: x.replace('%20', ' '))
df['text'] = df['text'].apply(lambda x: x.replace('@', ''))
train, val = train_test_split(df, stratify=df['target'])

test = pd.read_csv('data/test.csv')
test = test.fillna('')
test['text'] = test['text'].apply(lambda x: x.replace('%20', ' '))
test['text'] = test['text'].apply(lambda x: x.replace('@', ''))

In [42]:
def doc_to_token_vectors(text):
    doc = nlp(text)
    vecs = [token.vector for token in doc if sum(token.vector != 0.0)]
    return vecs

In [43]:
train['token_vectors'] = train['text'].apply(doc_to_token_vectors)
val['token_vectors'] = val['text'].apply(doc_to_token_vectors)
test['token_vectors'] = test['text'].apply(doc_to_token_vectors)

C:\Users\palme\Anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\palme\Anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
train.head()

,id,keyword,location,text,target,token_vectors
6632,9499,terrorist,,HereÛªs how media in Pakistan covered the cap...,1,"[[-0.23205, 0.47468, -0.38264, 0.0022248, -0.1..."
7457,10672,wounds,"Wales, United Kingdom",BritishBakeOff This has opened up old baked al...,0,"[[-0.087595, 0.35502, 0.063868, 0.29292, -0.23..."
4698,6679,landslide,,Method in contemplation of incident an leading...,1,"[[-0.35143, 0.79425, -0.16878, -0.11695, -0.47..."
6883,9868,traumatised,Scotland,PyramidHead76 one good thing came out of watch...,1,"[[0.054071, 0.1111, -0.14557, -0.024294, 0.381..."
1197,1723,buildings%20burning,"Mackay, QLD, Australia",Mmmmmm I'm burning.... I'm burning buildings I...,1,"[[0.59189, -0.29873, -0.28392, -0.36935, 0.157..."


In [41]:
sum(train.iloc[0]['token_vectors'][3])

2.132869705092162

In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import numpy as np

In [45]:
pad_value = np.array([0.0] *300)

X_train = pad_sequences(train['token_vectors'], padding='post', truncating='post', value=pad_value)
X_val = pad_sequences(val['token_vectors'], len(X_train[0]), padding='post', truncating='post', value=pad_value)
X_test = pad_sequences(test['token_vectors'], len(X_train[0]), padding='post', truncating='post', value = pad_value)

In [46]:
X_test.shape

(3263, 72, 300)

In [47]:
model = Sequential()
model.add(LSTM(300, input_shape=X_train.shape[1:], return_sequences=True))
model.add(Dropout(.2))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
model.fit(X_train, train['target'], epochs=10, validation_data=(X_val, val['target']))

Epoch 1/10
179/179 [==============================] - 114s 609ms/step - loss: 0.6954 - accuracy: 0.5586 - val_loss: 0.6837 - val_accuracy: 0.5704
Epoch 2/10
179/179 [==============================] - 125s 699ms/step - loss: 0.6868 - accuracy: 0.5610 - val_loss: 0.6714 - val_accuracy: 0.5704
Epoch 3/10
179/179 [==============================] - 120s 673ms/step - loss: 0.6851 - accuracy: 0.5652 - val_loss: 0.6843 - val_accuracy: 0.5704
Epoch 4/10
 95/179 [==============>...............] - ETA: 48s - loss: 0.6875 - accuracy: 0.5618

KeyboardInterrupt: 